In [1]:
import torch
import cv2
import json
from collections import deque
import matplotlib.pyplot as plt
from datetime import datetime

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Open video
cap = cv2.VideoCapture("sample_video.mp4")
if not cap.isOpened():
    raise IOError("Could not open video file.")

# Read first frame to get properties
ret, sample_frame = cap.read()
if not ret:
    raise IOError("Failed to read the first frame.")

height, width = sample_frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter("output_with_alerts.avi", fourcc, 20.0, (width, height))

# Reset pointer to start from frame 0 again
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

Using cache found in /home/steel/.cache/torch/hub/ultralytics_yolov5_master
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
YOLOv5 🚀 2025-7-12 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


True

In [2]:
frame_counter = 0
recent_counts = deque(maxlen=5)
alert_timestamps = []
alert_log = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1

    # Process every 3rd frame
    if frame_counter % 3 == 0:
        # Run detection
        results = model(frame)
        df = results.pandas().xyxy[0]
        persons = df[df['name'] == 'person']

        count = len(persons)
        recent_counts.append(count)

        # Check alert condition
        if len(recent_counts) == 5 and all(c >= 3 for c in recent_counts):
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            alert_msg = f"Crowd Detected at frame {frame_counter} ({timestamp})"
            print(alert_msg)

            # Save to log
            alert_timestamps.append(frame_counter)
            alert_log.append({"frame": frame_counter, "timestamp": timestamp, "count": count})

            # Overlay alert text
            cv2.putText(
                frame, "ALERT: Crowd Detected!", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3
            )

        # Draw boxes
        for _, row in persons.iterrows():
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Write frame to output video
    out.write(frame)

    # Show frame
    cv2.imshow("Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Save alerts to JSON
with open("alerts_log.json", "w") as f:
    json.dump(alert_log, f, indent=2)

# Plot timeline
plt.figure(figsize=(10, 4))
plt.plot(alert_timestamps, [1]*len(alert_timestamps), 'ro')
plt.xlabel("Frame Number")
plt.ylabel("Alert Occurrence")
plt.title("Timeline of Crowd Alerts")
plt.yticks([])
plt.tight_layout()
plt.savefig("alerts_timeline.png")
plt.show()

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()

/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906:

Crowd Detected at frame 69 (2025-07-13 09:26:43)
Crowd Detected at frame 72 (2025-07-13 09:26:43)
Crowd Detected at frame 75 (2025-07-13 09:26:43)
Crowd Detected at frame 78 (2025-07-13 09:26:43)
Crowd Detected at frame 81 (2025-07-13 09:26:43)


/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906:

Crowd Detected at frame 84 (2025-07-13 09:26:43)


/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906:

...

/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906:

Crowd Detected at frame 2004 (2025-07-13 09:27:12)
Crowd Detected at frame 2007 (2025-07-13 09:27:12)
Crowd Detected at frame 2010 (2025-07-13 09:27:12)
Crowd Detected at frame 2013 (2025-07-13 09:27:12)
Crowd Detected at frame 2016 (2025-07-13 09:27:12)


/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906:

Crowd Detected at frame 2019 (2025-07-13 09:27:12)


/home/steel/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
